In [70]:
suppressPackageStartupMessages({
    library(jsonlite)
    library(tidyverse)
    library(lubridate)
    library(Amelia)
    library(ggplot2)
    library(plotly)
    library(magrittr)
    library(ggrepel)
    library(repr)
    library(gridExtra)
})

In [71]:
data <- jsonlite::fromJSON("https://op-koti.fi/api/apartments?mode=sale&featureGroup=apartment&orderBy=created&order=desc&offset=0&limit=4500", flatten = T)

df <- data[[2]]

In [72]:
cols = c('id', 'listingType', 'floor', 'year', 'rooms', 'numberOfRooms', 'price', 'debtFreePrice',
        'location.city', 'location.region', 'location.district','location.postalCode',
        'livingArea.size', 'totalArea.size')

df <- df[cols]

df <- df %>%
  rename(city = location.city, 
         region = location.region, 
         district = location.district, 
         postalCode = location.postalCode, 
         livingArea = livingArea.size, 
         totalArea = totalArea.size, 
         yearBuilt = year)

In [73]:
head(df)

,id,listingType,floor,yearBuilt,rooms,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea
,<chr>,<int>,<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,520650,90,0,1968,"4h,k,kph,khh,s",4,85000.0,NA,Salo,Salo,Teijo,25570,95.0,178.0
2,520719,89,3,1974,"2h, k",2,25660.0,45000,Keuruu,Keuruu,Kivelä,42700,55.0,55.0
3,520787,89,5,1966,"1h,kk",1,67500.0,67500,Rauma,Rauma,Nummi,26100,32.5,32.5
4,520143,113,2,1981,"3h,k,kph",3,187477.4,220000,Helsinki,Länsi-Helsinki,Lassila,00440,72.5,72.5
5,518671,89,3,2008,"2h,k,s",2,128000.0,128000,Salo,Salo,Moisio,24100,43.5,43.5
6,520722,89,4,1970,"3h,k",3,114769.8,125000,Uusikaupunki,Uusikaupunki,Sorvakko,23500,78.0,78.0


listingType is in code. The code below changes it into the corresponding text values

In [74]:
df <- df %>%
  mutate(listingType = recode(listingType, '89'='Kerrostalo','90'='Omakotitalo','91'='Rivitalo','92'='Paritalo','93'='Erillistalo','112'='Puutalo','113'='Luhtitalo','470'='Kytketty paritalo'))

In [75]:
table(df$listingType)


      Erillistalo        Kerrostalo Kytketty paritalo         Luhtitalo 
               14              1483                 1                45 
      Omakotitalo          Paritalo           Puutalo          Rivitalo 
              909               105                 3               650 

replacing empty values with NAs

In [76]:
df <- df %>% 
  mutate_all(na_if,"")

df %>%
  summarise_all(funs(sum(is.na(.))))

id,listingType,floor,yearBuilt,rooms,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
0,0,532,7,115,115,114,1073,0,0,188,0,115,115


#### Next,
* Cases where debtFreePrice is NA but price is given, the price of the listing is the debtFreePrice. This means there is no outstanding loan payment to be carried over to the new owner

* Also creating a new row 'buildingAge' from the 'yearBuilt'

In [77]:
df <- df %>%
  mutate(debtFreePrice = case_when(is.na(debtFreePrice) & !is.na(price) ~ price,
                                   TRUE ~ debtFreePrice))

df <- df %>%
  filter(!is.na(yearBuilt))%>%
  mutate(buildingAge = case_when(yearBuilt > year(today()) ~ 1,
                                 TRUE ~ year(today()) - year(as.Date(as.character(yearBuilt), format = "%Y")))) %>%
  select(-yearBuilt)

Removing the NAs

In [78]:
df <- na.omit(df)

In [79]:
## CREATED FUNCTIONS
is.not.null <- function(x) !is.null(x)

clean_rooms <- function(rooms){
  clean = c()
  for (room in rooms){
    room = str_replace(room, "\\.", " ")
    y = strsplit(room, split = "[[:punct:]]")
    z = c()
    for (x in y) {
      z = append(z, str_trim(x, side = "both"))
    }
    z = z[!z == ""]
    # y <- paste(z, collapse = " ", recycle0 = FALSE)
    y = list(z)
    clean = append(clean, y)
  }
  return(clean)
}

detect_sauna <- function(rooms){
  # takes a list of rooms, checks for sauna and returns binary
  l = c()
  for(room in rooms){
    l = append(l,any(str_detect(room, '^s$|sauna')))
  }
  return(as.integer(l))
}

detect_balcony <- function(rooms){
  # takes a list of rooms, checks for balcony and returns binary
  l = c()
  for(room in rooms){
    l = append(l,any(str_detect(room, '^p$|parv|lasit p|las p')))
  }
  return(as.integer(l))
}

detect_parking <- function(rooms){
  # takes a list of rooms, checks for parking and returns binary
  l = c()
  for(room in rooms){
    l = append(l,any(str_detect(room, '^ak$|^at$|auto')))
  }
  return(as.integer(l))
}

detect_walk_in_closet <- function(rooms){
  # takes a list of rooms, checks for walk-in-closet and returns binary
  l = c()
  for(room in rooms){
    l = append(l,any(str_detect(room, '^v$|^vh$|vaate')))
  }
  return(as.integer(l))
}

detect_storage <- function(rooms){
  # takes a list of rooms, checks for storage room and returns binary
  l = c()
  for(room in rooms){
    l = append(l,any(str_detect(room, 'var')))
  }
  return(as.integer(l))
}

In [80]:
head(df$rooms,20)

[1] "4h,k,kph,khh,s"                                    
 [2] "2h, k"                                             
 [3] "1h,kk"                                             
 [4] "3h,k,kph"                                          
 [5] "2h,k,s"                                            
 [6] "3h,k"                                              
 [7] "2h,kk"                                             
 [8] "2h,k,kph"                                          
 [9] "2h,k"                                              
[10] "2h,k,s,kph,wc,2vh,var"                             
[11] "2h, kt, s"                                         
[12] "2h,k"                                              
[13] "2h, k, s, var"                                     
[14] "3mh,oh,k,kph,s"                                    
[15] "3h,k,s"                                            
[16] "4h+k+ph+s"                                         
[17] "5h+k+"                                             
[18] "2h+kk"                                             
[19] "1h,kk"                                             
[20] "5h,k,kph,khh,ph,s,2 tilaa/varasto, 2 vh,at/varasto"

We can see that the rooms column has information in varying formats. clean_rooms() makes it uniform

In [81]:
df$rooms = clean_rooms(df$rooms)
head(df$rooms,20)

[[1]]
[1] "4h"  "k"   "kph" "khh" "s"  

[[2]]
[1] "2h" "k" 

[[3]]
[1] "1h" "kk"

[[4]]
[1] "3h"  "k"   "kph"

[[5]]
[1] "2h" "k"  "s" 

[[6]]
[1] "3h" "k" 

[[7]]
[1] "2h" "kk"

[[8]]
[1] "2h"  "k"   "kph"

[[9]]
[1] "2h" "k" 

[[10]]
[1] "2h"  "k"   "s"   "kph" "wc"  "2vh" "var"

[[11]]
[1] "2h" "kt" "s" 

[[12]]
[1] "2h" "k" 

[[13]]
[1] "2h"  "k"   "s"   "var"

[[14]]
[1] "3mh" "oh"  "k"   "kph" "s"  

[[15]]
[1] "3h" "k"  "s" 

[[16]]
[1] "4h" "k"  "ph" "s" 

[[17]]
[1] "5h" "k" 

[[18]]
[1] "2h" "kk"

[[19]]
[1] "1h" "kk"

[[20]]
 [1] "5h"      "k"       "kph"     "khh"     "ph"      "s"       "2 tilaa"
 [8] "varasto" "2 vh"    "at"      "varasto"

In the next phase we extract features of each listing from the 'rooms' column

In [82]:
df <- df %>%
  mutate(centrum = case_when(str_detect(postalCode, "100$") ~ as.integer(1),
                             TRUE ~ as.integer(0)))

df$hasSauna <- detect_sauna(df$rooms)
df$hasBalcony <- detect_balcony(df$rooms)
df$hasParking <- detect_parking(df$rooms)
df$hasWalkInCloset <- detect_walk_in_closet(df$rooms)
df$hasStorage <- detect_storage(df$rooms)

finally, the 'rooms' column can be removed

In [83]:
df <- df %>%
  select(-rooms)

In [84]:
# Creating new variables for price per meter square and link to the respective houses on the website
df <- df %>%
mutate(pricePMsq = debtFreePrice/totalArea, link = paste0("<a href='https://op-koti.fi/kohde/",id,"'>","https://op-koti.fi/kohde/",id,"</a>"))

In [85]:
table(df$listingType)


      Erillistalo        Kerrostalo Kytketty paritalo         Luhtitalo 
               11              1345                 1                43 
      Omakotitalo          Paritalo           Puutalo          Rivitalo 
              509                81                 2               552 

In [86]:
# Merging 'Kytketty paritalo' into 'Paritalo'
df$listingType[df$listingType %in% "Kytketty paritalo"] <- "Paritalo"
table(df$listingType)


Erillistalo  Kerrostalo   Luhtitalo Omakotitalo    Paritalo     Puutalo 
         11        1345          43         509          82           2 
   Rivitalo 
        552 

'Kytketty paritalo' is now merged into 'Paritalo'

Next we rearrange the columns for convenience

In [87]:
as.list(colnames(df))

[[1]]
[1] "id"

[[2]]
[1] "listingType"

[[3]]
[1] "floor"

[[4]]
[1] "numberOfRooms"

[[5]]
[1] "price"

[[6]]
[1] "debtFreePrice"

[[7]]
[1] "city"

[[8]]
[1] "region"

[[9]]
[1] "district"

[[10]]
[1] "postalCode"

[[11]]
[1] "livingArea"

[[12]]
[1] "totalArea"

[[13]]
[1] "buildingAge"

[[14]]
[1] "centrum"

[[15]]
[1] "hasSauna"

[[16]]
[1] "hasBalcony"

[[17]]
[1] "hasParking"

[[18]]
[1] "hasWalkInCloset"

[[19]]
[1] "hasStorage"

[[20]]
[1] "pricePMsq"

[[21]]
[1] "link"

In [88]:
df <- df[c(1,2,4,5,6,11,12,20,7:10,3,13:19,21)]

#### Now the df is ready for the dashboard. Let's save it to a csv file.

In [89]:
write.csv(df, "/Users/avinashmalla/GitHub/opKotiDashboard/forDash.csv", row.names = F)